In [ ]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

# external CSS stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


###########################
# Data Manipulation / Model
###########################

# Database connection parameters
username = "aacuser", 
password = "kxMi8CPhAJQj"
collection = "animals"
port = 46319
db = "AAC"
host = "localhost"
shelter = AnimalShelter("localhost", 46319, "AAC", "animals", "aacuser", "kxMi8CPhAJQj")
logo = 'assets/Grazioso Salvare Logo.png'

# class read method must support return of cursor object and accept projection json input
data = shelter.read({})

# Names to show for each column
cols = dict({'animal_id':'ID', 'animal_type':'Type', 'name':'Name', 'breed':'Breed', 'color': 'Color', 'date_of_birth': 'DOB', 'outcome_type':'Outcome', 'age_upon_outcome': 'Age', 'location_long': 'Longitude', 'location_lat': 'Latitude'})
df = pd.DataFrame.from_records(data, columns=['animal_id', 'animal_type', 'name', 'breed', 'color', 'date_of_birth', 'outcome_type', 'age_upon_outcome', 'location_long', 'location_lat'])
initial_active_cell = {"row": 0, "column": 0, "column_id": "animal_id", "row_id": 0}

# Reference to the external stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#########################
# Dashboard Layout / View
#########################

# Instantiation of app variable that references an external stylesheet
app = JupyterDash('Module 5', external_stylesheets=external_stylesheets)

# Rules to apply conditional styles 
# to table
style_data_conditional = [
    {
        'if': {'row_index': 'odd'}, 
        'backgroundColor':'#E8F8FF !important',
    },
    
]

# Application layout to show all elements it html format
app.layout = html.Div(
    [
        html.Div(
            [
                html.Center(html.Img(src=logo, height=100, width=100)),
                html.H3("Eric Wallace SNHU CS-340 Project Two", style={"textAlign":"center", 'color': '#176EA8'}),
                dcc.Dropdown(id='rescue-filter',
                    options=[
                        {"label":'Water Rescue', "value": 'water'}, 
                        {"label":'Mountain Rescue', "value": 'mountain'},
                        {"label":'Disaster Rescue', "value": 'disaster'},
                        {"label":'Reset', "value": 'reset'},
                    ],
                    optionHeight=35,
                    multi = False,
                    searchable = True,
                    disabled = False,
                    placeholder = "Select a rescue group",
                    style={'width': '60%', 'font-size': 14, 'margin-bottom': 10},
                ),
                dash_table.DataTable(
                    id="table",
                    columns=[{"name": cols[c], "id": c} for c in df.columns],
                    data=df.to_dict("records"),
                    page_size=10,
                    page_current=0,
                    page_action='native',
                    sort_action="native",
                    filter_action="native",
                    hidden_columns=['location_long', 'location_lat', 'animal_id'],
                    active_cell=initial_active_cell,
                    style_cell = {"fontFamily": "Roboto", "size": 9, 'textAlign': 'center'},
                    style_header={'backgroundColor': '#176EA8', 'color': 'white', 'padding': '5px', 'textAlign': 'center'},
                    style_data_conditional=style_data_conditional,
                    css=[
                        {'selector':'.dash-spreadsheet-menu-item', 'rule': 'display:none'},
                    ]
                ),
            ],
            className="eleven columns row"
        ),
        html.Div(
            [
                html.Div(dcc.Graph(id='output-chart'), className="six columns"),
                html.Div(id="output-map", className="five columns"),
            ],
            className="eleven columns row",
            style={"border": "1px solid #176EA8", "position": "relative", "left": 0, "margin-left": 0, "margin-top": 10}
        ),
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################
# Callback takes in the active cell and then
# outputs to the conditional
@app.callback(
    Output('table', 'style_data_conditional'),
    [Input('table', 'active_cell')],
)

# Function takes in the active cell
# and then based on the selection it
# highlights the row in which the cell
# is selected
def update_styles(active_cell):
    if active_cell:
        row = active_cell['row']
        animal_id = df.at[row, 'animal_id']
        return [{
            'if': { 'row_index' : row},
            'background_color': "#ffe091",
            'border': '1px solid #176EA8',
        }]

# Callback takes in the active cell
# and outputs the map to the output-map
@app.callback(
    Output('output-map', "children"),
    [Input("table", 'active_cell')]
)

# Function that is responsible for updating the map
# this is accomplished by looking up the active cell
# and returning the location of that row of data
def update_map(active_cell):
    if active_cell is None:
        raise PreventUpdate
    row = active_cell['row']
    long = df.at[row, 'location_long']
    lat = df.at[row, 'location_lat']
    name = df.at[row, 'name'],
    animal_type = df.at[row, 'animal_type']
    
    return [
        dl.Map(style={'width': '600px', 'height': '500px', 'margin-top': 0}, center=[lat,long], zoom=9, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat,long], children=[
                dl.Tooltip(animal_type),
                dl.Popup([
                    html.H6(name)
                ])
            ])
        ])
    ]

# Callback function used to filter
# data by rescue type
@app.callback(
    Output('table', 'data'),
    [Input('rescue-filter', 'value')]
)

# Function used to sort the data based
# on the selection in the dropdown
def filter_table(value):
    if not value:
        raise PreventUpdate
    else:
        disaster_breeds = { "animal_type": "Dog", "breed": { "$in": ['Doberman Pinscher','German Shepherd','Golden Retriever','Bloodhound', 'Rottweiler']}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 300}},
        mountain_breeds = { "animal_type": "Dog", "breed": { "$in": ["German Shepherd", "Alaskan Malamute", "Old English", "Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 156}},
        water_breeds = { "animal_type": "Dog", "breed": { "$in": ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 156}},
        
        if value == 'water':
            data = shelter.read({ "animal_type": "Dog", "breed": { "$in": ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 156}})
        elif value == 'mountain':
            data = shelter.read({ "animal_type": "Dog", "breed": { "$in": ["German Shepherd", "Alaskan Malamute", "Old English", "Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 156}})
        elif value == 'disaster':
            data = shelter.read({ "animal_type": "Dog", "breed": { "$in": ['Doberman Pinscher','German Shepherd','Golden Retriever','Bloodhound', 'Rottweiler']}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 300}})
        elif value == 'reset':
            data = shelter.read({})
            
        dff = pd.DataFrame.from_records(data, columns=['animal_id', 'animal_type', 'name', 'breed', 'color', 'date_of_birth', 'outcome_type', 'age_upon_outcome', 'location_long', 'location_lat'])
        
        return dff.to_dict("records")

# Callback takes the output of the
# previous callback and processes
# the data to output a pie chart
@app.callback(
    Output('output-chart', 'figure'),
    [Input('table', 'derived_viewport_data')],
)

# Function creates the pie chart 
# based on information provided
# by previous callback
def update_graph(data):
    if data is None:
        raise PreventUpdate
    else:
        piechart = px.pie(
            data,
            labels="breed",
            names="breed",
        )
        return piechart
    
if __name__ == '__main__':
    app.run_server(mode="external")